In [1]:
from math import sqrt
from numpy import concatenate
import numpy as np
from numpy import array

import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.utils import to_categorical
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD
import tensorflow as tf

from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from matplotlib import pyplot
from sodapy import Socrata


In [ ]:
# # Uncomment the below to open data source, download sensor data, and store it as a csv locally.

# #Function to get Sensor count history data
# def sensor_count():
#     client = Socrata('data.melbourne.vic.gov.au', 'nlPM0PQJSjzCsbVqntjPvjB1f', None)
#     sensor_data_id = "b2ak-trbp"
#     results = client.get(sensor_data_id, limit=5000000)
#     df = pd.DataFrame.from_records(results)
#     df = df[['date_time', 'year', 'month', 'mdate', 'day', 'time', 'sensor_id', 'sensor_name', 'hourly_counts']]
#     return df

# sensor_history = sensor_count()

# sensor_history.to_csv('sensor_history.csv', index=False)

In [2]:
sensor_history = pd.read_csv('sensor_history.csv')

In [3]:
single_sensor = sensor_history[sensor_history.sensor_id == 4].copy()
single_sensor.sort_values('date_time', inplace=True, ascending=True)

In [ ]:
#Make sensor counts range from 0 to 1.
maximum = single_sensor.hourly_counts.max()
single_sensor['hourly_counts'] = single_sensor['hourly_counts']/maximum

Now, it is important to understand that a model that attempts to make predictions on a time series can have reasonably good success by simply copying the value of the previous time step. In most cases, this value will be close to the next value in the sequence - and this can give the appearance of predictive ability where there is actually none.

So before building models and evaluating them, it is worthwhile setting up a baseline. This is the performance attained by simply copying the hourly count from the previous time step.

In [ ]:
true_count = single_sensor.hourly_counts.to_numpy()
prev_count = true_count[1:true_count.shape[0]]
r2_baseline = r2_score(prev_count, true_count[0:true_count.shape[0]-1])   
print("The baseline R-squared score is:", r2_baseline)

From this example we can see a fairly convincing score (at the time of writing this comment, it was around 0.884). However, we know that the model we just made is useless for our purposes.
We need to improve on this score!

In [ ]:
sensor_seq = []
for i in range(0, len(single_sensor)):
    sensor_seq.append(single_sensor.hourly_counts.iloc[i])

In [ ]:
# split a univariate sequence
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# choose a number of time steps
n_steps = 392
# split into samples
X, y = split_sequence(sensor_seq, n_steps)


In [ ]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))


In [ ]:
X_train = X[10001:80000]
X_val = X[80001:]
X_test = X[:10000]
Y_train = y[10001:80000]
Y_val = y[80001:]
Y_test = y[:10000]

In [ ]:
epochs = 50
learning_rate = 0.00001
decay_rate = learning_rate / epochs
momentum = 0.8

# define model
model = Sequential()
model.add(LSTM(48, return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(96, return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(120, return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(196, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(Dense(48, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(optimizer=opt, loss='mse')


In [ ]:
# fit model
model.fit(X_train, Y_train, validation_data =(X_val, Y_val), epochs=45, verbose=1, batch_size = 64)


In [ ]:
# demonstrate prediction
x_input = array(sensor_seq[10001:10393])
# x_input = x_input/maximum
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print("predicted next value: ", yhat * maximum)
print("actual next value: ", sensor_seq[10393] * maximum)

In [ ]:
# demonstrate prediction
x_input = array(sensor_seq[10021:10413])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print("predicted next value: ", yhat * maximum)
print("actual next value: ", sensor_seq[10413] * maximum)

In [ ]:
pred = model.predict(X_test)

In [ ]:
r2_score(Y_test, pred)

In [ ]:
# model.save("model_3.h5")

In [ ]:
# #Function to get Sensor count history data
# def micro_count():
#     client = Socrata('data.melbourne.vic.gov.au', 'nlPM0PQJSjzCsbVqntjPvjB1f', None)
#     micro_data_id = "u4vh-84j8"
#     results = client.get(micro_data_id, limit=4000000)
#     if results:
#         df = pd.DataFrame.from_records(results)
#     return df

# micro_history = micro_count()

# micro_history.to_csv('micro_history.csv', index=False)

In [4]:
micro_history = pd.read_csv('micro_history.csv')

micro_history = micro_history[(micro_history.sensor_id == '5a') | (micro_history.sensor_id == '5b') |
                             (micro_history.sensor_id == '5c') | (micro_history.sensor_id == '0a') |
                             (micro_history.sensor_id == '0b') | (micro_history.sensor_id == '6')]

micro_history = micro_history[(micro_history.site_id == 1003) | (micro_history.site_id == 1009)]

micro_history = micro_history.drop(['id', 'gateway_hub_id', 'type', 'units'], axis=1)

micro_history.loc[micro_history.sensor_id == '5a', 'temp'] = micro_history.value
micro_history.loc[micro_history.sensor_id == '5b', 'humidity'] = micro_history.value
micro_history.loc[micro_history.sensor_id == '5c', 'pressure'] = micro_history.value
micro_history.loc[micro_history.sensor_id == '0a', 'part_2p5'] = micro_history.value
micro_history.loc[micro_history.sensor_id == '0b', 'part_10'] = micro_history.value
micro_history.loc[micro_history.sensor_id == '6', 'wind'] = micro_history.value

micro_history.local_time = pd.to_datetime(micro_history.local_time, format='%Y-%m-%d')
micro_history['year'] = micro_history.local_time.dt.year
micro_history['month'] = micro_history.local_time.dt.month_name()
micro_history['mdate'] = micro_history.local_time.dt.day
micro_history['time'] = micro_history.local_time.dt.hour

micro_history = micro_history.drop(['site_id', 'sensor_id', 'value', 'local_time'], axis=1)
micro_history = micro_history.groupby(by=['year', 'month', 'mdate', 'time']).max()

ped_climate = sensor_history.merge(micro_history, on=('year', 'month', 'mdate', 'time'), how='inner')

In [45]:
single_sensor_climate = ped_climate.loc[ped_climate.sensor_name == 'Town Hall (West)'].copy()

In [46]:
single_sensor_climate.fillna(method='ffill', inplace = True)

In [47]:
single_sensor_climate.head()

,date_time,year,month,mdate,day,time,sensor_id,sensor_name,hourly_counts,temp,humidity,pressure,part_2p5,part_10,wind
512,2019-11-15T09:00:00.000,2019,November,15,Friday,9,4,Town Hall (West),1430,15.41,66.14,1008.0,4.5,18.4,7.92
567,2019-11-15T10:00:00.000,2019,November,15,Friday,10,4,Town Hall (West),2101,16.12,59.41,1008.0,4.2,20.0,7.88
622,2019-11-15T11:00:00.000,2019,November,15,Friday,11,4,Town Hall (West),2577,16.90,57.53,1007.9,4.1,20.1,7.92
677,2019-11-15T12:00:00.000,2019,November,15,Friday,12,4,Town Hall (West),3439,18.73,54.64,1007.5,4.5,23.1,9.68
732,2019-11-15T13:00:00.000,2019,November,15,Friday,13,4,Town Hall (West),4043,18.78,51.06,1006.7,4.1,18.7,10.44


In [48]:
single_sensor_climate.drop(['date_time', 'year', 'sensor_id', 'sensor_name', 'mdate'], axis = 1, inplace = True)

In [55]:
single_sensor_climate.head()

,month,day,time,hourly_counts,temp,humidity,pressure,part_2p5,part_10,wind
512,11,5,9,1430,15.41,66.14,1008.0,4.5,18.4,7.92
567,11,5,10,2101,16.12,59.41,1008.0,4.2,20.0,7.88
622,11,5,11,2577,16.90,57.53,1007.9,4.1,20.1,7.92
677,11,5,12,3439,18.73,54.64,1007.5,4.5,23.1,9.68
732,11,5,13,4043,18.78,51.06,1006.7,4.1,18.7,10.44


In [50]:
month_dict = {'January' : 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7,
              'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}
day_dict = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}

In [51]:
single_sensor_climate.replace({'month': month_dict, 'day': day_dict}, inplace = True)

In [52]:
scaler = MinMaxScaler()

In [53]:
scaled = scaler.fit_transform(single_sensor_climate)

In [33]:
new_sequence = np.zeros((scaled.shape[0], scaled.shape[1]+1), dtype=float)

In [54]:
for i in range (0, scaled.shape[0]-1):
    new_sequence[i][0:9] = scaled[i][0:9]
    new_sequence[i][10] = scaled[i+1][3]

In [55]:
train = new_sequence[0:12000, :]
test = new_sequence[12000:15000, :]
val = new_sequence[15000:,:]

In [56]:
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
val_x, val_y = val[:, :-1], val[:, -1]

In [57]:
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))
val_x = val_x.reshape((val_x.shape[0], 1, val_x.shape[1]))

In [21]:
epochs = 50
learning_rate = 0.00001
decay_rate = learning_rate / epochs
momentum = 0.8

# define model
model = Sequential()
model.add(LSTM(48, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(96, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(120, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(196, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(48, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(optimizer=opt, loss='mse')

In [59]:
model.fit(train_x, train_y, validation_data =(test_x, test_y), epochs=1000, verbose=1, batch_size = 64)

Epoch 1/1000
188/188 [==============================] - 4s 20ms/step - loss: 0.1640 - val_loss: 0.1584
Epoch 2/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.1628 - val_loss: 0.1570
Epoch 3/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.1611 - val_loss: 0.1549
Epoch 4/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.1585 - val_loss: 0.1515
Epoch 5/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.1542 - val_loss: 0.1458
Epoch 6/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.1468 - val_loss: 0.1359
Epoch 7/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.1344 - val_loss: 0.1195
Epoch 8/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.1152 - val_loss: 0.0955
Epoch 9/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0899 - val_loss: 0.0672
Epoch 10/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.06

Epoch 159/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0045 - val_loss: 0.0054
Epoch 160/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0045 - val_loss: 0.0054
Epoch 161/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0044 - val_loss: 0.0054
Epoch 162/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0045 - val_loss: 0.0054
Epoch 163/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0044 - val_loss: 0.0054
Epoch 164/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0045 - val_loss: 0.0054
Epoch 165/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0044 - val_loss: 0.0054
Epoch 166/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0044 - val_loss: 0.0054
Epoch 167/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0043 - val_loss: 0.0054
Epoch 168/1000
188/188 [==============================] - 2s 9ms

Epoch 316/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0038 - val_loss: 0.0050
Epoch 317/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0038 - val_loss: 0.0050
Epoch 318/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0037 - val_loss: 0.0050
Epoch 319/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0037 - val_loss: 0.0050
Epoch 320/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0038 - val_loss: 0.0050
Epoch 321/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0038 - val_loss: 0.0050
Epoch 322/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0037 - val_loss: 0.0050
Epoch 323/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0037 - val_loss: 0.0050
Epoch 324/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0037 - val_loss: 0.0050
Epoch 325/1000
188/188 [==============================] - 2s 9ms

Epoch 395/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0036 - val_loss: 0.0049
Epoch 396/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0036 - val_loss: 0.0049
Epoch 397/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0036 - val_loss: 0.0049
Epoch 398/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0036 - val_loss: 0.0049
Epoch 399/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0035 - val_loss: 0.0049
Epoch 400/1000
188/188 [==============================] - 2s 10ms/step - loss: 0.0036 - val_loss: 0.0049
Epoch 401/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0036 - val_loss: 0.0049
Epoch 402/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0036 - val_loss: 0.0049
Epoch 403/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0036 - val_loss: 0.0049
Epoch 404/1000
188/188 [==============================] - 2s 9m

Epoch 552/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 553/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 554/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 555/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 556/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 557/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 558/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 559/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 560/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 561/1000
188/188 [==============================] - 2s 9ms

Epoch 709/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0031 - val_loss: 0.0047
Epoch 710/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0031 - val_loss: 0.0047
Epoch 711/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0031 - val_loss: 0.0047
Epoch 712/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0031 - val_loss: 0.0047
Epoch 713/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0031 - val_loss: 0.0046
Epoch 714/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0031 - val_loss: 0.0047
Epoch 715/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0031 - val_loss: 0.0047
Epoch 716/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0031 - val_loss: 0.0047
Epoch 717/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0031 - val_loss: 0.0046
Epoch 718/1000
188/188 [==============================] - 2s 8ms

Epoch 866/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0030 - val_loss: 0.0047
Epoch 867/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0047
Epoch 868/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0046
Epoch 869/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0047
Epoch 870/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0047
Epoch 871/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0031 - val_loss: 0.0047
Epoch 872/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0031 - val_loss: 0.0047
Epoch 873/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0047
Epoch 874/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0030 - val_loss: 0.0047
Epoch 875/1000
188/188 [==============================] - 2s 8ms

Epoch 945/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0047
Epoch 946/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0047
Epoch 947/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0047
Epoch 948/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0047
Epoch 949/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 950/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0047
Epoch 951/1000
188/188 [==============================] - 2s 9ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 952/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0047
Epoch 953/1000
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0047
Epoch 954/1000
188/188 [==============================] - 2s 9ms

In [60]:
yhat = model.predict(val_x)
val_x = val_x.reshape((val_x.shape[0], val_x.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, val_x[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
val_y = val_y.reshape((len(val_y), 1))
inv_y = concatenate((val_y, val_x[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

r2 = r2_score(inv_y, inv_yhat)
print('R-squared score: %.3f' % r2)

R-squared score: 0.936


In [61]:
#What about if we try one hot encoding instead?
single_sensor_climate = ped_climate.loc[ped_climate.sensor_name == 'Town Hall (West)'].copy()
single_sensor_climate.fillna(method='ffill', inplace = True)

month_dummies = pd.get_dummies(single_sensor_climate.month)
day_dummies = pd.get_dummies(single_sensor_climate.day)

In [62]:
single_sensor_climate.drop(['date_time', 'month', 'day', 'year', 'sensor_id', 'sensor_name', 'mdate'], axis = 1, inplace = True)

In [63]:
single_sensor_climate = pd.concat([single_sensor_climate, month_dummies, day_dummies], axis = 1)

In [64]:
single_sensor_climate

,time,hourly_counts,temp,humidity,pressure,part_2p5,part_10,wind,April,August,...,November,October,September,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
512,9,1430,15.41,66.14,1008.0,4.5,18.4,7.92,0,0,...,1,0,0,1,0,0,0,0,0,0
567,10,2101,16.12,59.41,1008.0,4.2,20.0,7.88,0,0,...,1,0,0,1,0,0,0,0,0,0
622,11,2577,16.90,57.53,1007.9,4.1,20.1,7.92,0,0,...,1,0,0,1,0,0,0,0,0,0
677,12,3439,18.73,54.64,1007.5,4.5,23.1,9.68,0,0,...,1,0,0,1,0,0,0,0,0,0
732,13,4043,18.78,51.06,1006.7,4.1,18.7,10.44,0,0,...,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146126,19,1641,16.49,55.05,1022.3,7.7,32.1,0.00,1,0,...,0,0,0,0,0,1,0,0,0,0
1146190,20,1239,16.26,54.96,1022.7,8.2,38.9,0.00,1,0,...,0,0,0,0,0,1,0,0,0,0
1146254,21,1083,15.92,55.24,1023.3,13.0,42.5,0.00,1,0,...,0,0,0,0,0,1,0,0,0,0
1146318,22,883,15.52,57.24,1023.6,7.0,32.2,0.00,1,0,...,0,0,0,0,0,1,0,0,0,0


In [68]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(single_sensor_climate)

new_sequence = np.zeros((scaled.shape[0], scaled.shape[1]+1), dtype=float)

for i in range (0, scaled.shape[0]-1):
    new_sequence[i][0:26] = scaled[i][0:26]
    new_sequence[i][27] = scaled[i+1][1]  


In [73]:
scaled[1]

array([4.34782609e-01, 4.04116176e-01, 3.44222916e-01, 5.56962025e-01,
       4.14342629e-01, 4.37876300e-04, 2.53623188e-03, 3.61799816e-01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [69]:
train = new_sequence[0:12000, :]
test = new_sequence[12000:15000, :]
val = new_sequence[15000:,:]

train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
val_x, val_y = val[:, :-1], val[:, -1]

train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))
val_x = val_x.reshape((val_x.shape[0], 1, val_x.shape[1]))

In [70]:
model.fit(train_x, train_y, validation_data =(test_x, test_y), epochs=900, verbose=1, batch_size = 64)

Epoch 1/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0375 - val_loss: 0.0249
Epoch 2/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0243 - val_loss: 0.0193
Epoch 3/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0218 - val_loss: 0.0174
Epoch 4/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0205 - val_loss: 0.0164
Epoch 5/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0195 - val_loss: 0.0157
Epoch 6/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0188 - val_loss: 0.0151
Epoch 7/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0181 - val_loss: 0.0147
Epoch 8/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0175 - val_loss: 0.0143
Epoch 9/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0170 - val_loss: 0.0139
Epoch 10/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0163 - val_lo

188/188 [==============================] - 2s 8ms/step - loss: 0.0041 - val_loss: 0.0052
Epoch 82/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0042 - val_loss: 0.0052
Epoch 83/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0041 - val_loss: 0.0052
Epoch 84/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0042 - val_loss: 0.0052
Epoch 85/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0041 - val_loss: 0.0052
Epoch 86/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0041 - val_loss: 0.0051
Epoch 87/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0041 - val_loss: 0.0051
Epoch 88/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0042 - val_loss: 0.0051
Epoch 89/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0041 - val_loss: 0.0051
Epoch 90/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0041 - val_loss: 

Epoch 161/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0037 - val_loss: 0.0049
Epoch 162/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0038 - val_loss: 0.0049
Epoch 163/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0037 - val_loss: 0.0049
Epoch 164/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0038 - val_loss: 0.0049
Epoch 165/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0037 - val_loss: 0.0049
Epoch 166/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0037 - val_loss: 0.0049
Epoch 167/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0038 - val_loss: 0.0049
Epoch 168/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0037 - val_loss: 0.0049
Epoch 169/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0037 - val_loss: 0.0049
Epoch 170/900
188/188 [==============================] - 2s 8ms/step - lo

188/188 [==============================] - 2s 8ms/step - loss: 0.0035 - val_loss: 0.0048
Epoch 241/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0035 - val_loss: 0.0048
Epoch 242/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0035 - val_loss: 0.0048
Epoch 243/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0036 - val_loss: 0.0048
Epoch 244/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0035 - val_loss: 0.0048
Epoch 245/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0035 - val_loss: 0.0048
Epoch 246/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0035 - val_loss: 0.0048
Epoch 247/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0035 - val_loss: 0.0048
Epoch 248/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0035 - val_loss: 0.0048
Epoch 249/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0035 - v

188/188 [==============================] - 2s 9ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 320/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 321/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 322/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 323/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 324/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 325/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0034 - val_loss: 0.0047
Epoch 326/900
188/188 [==============================] - 2s 9ms/step - loss: 0.0033 - val_loss: 0.0047
Epoch 327/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0034 - val_loss: 0.0047
Epoch 328/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0034 - v

...

188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 557/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 558/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 559/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0031 - val_loss: 0.0046
Epoch 560/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 561/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 562/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 563/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 564/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 565/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - v

188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 636/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 637/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 638/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 639/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 640/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0046
Epoch 641/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0046
Epoch 642/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 643/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0046
Epoch 644/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - v

188/188 [==============================] - 2s 8ms/step - loss: 0.0030 - val_loss: 0.0045
Epoch 715/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0046
Epoch 716/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0045
Epoch 717/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0046
Epoch 718/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0046
Epoch 719/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0046
Epoch 720/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0046
Epoch 721/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0046
Epoch 722/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0046
Epoch 723/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - v

188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 794/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0046
Epoch 795/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0046
Epoch 796/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0046
Epoch 797/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0046
Epoch 798/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 799/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0029 - val_loss: 0.0046
Epoch 800/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 801/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 802/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - v

188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 873/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 874/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 875/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 876/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 877/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 878/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 879/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 880/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 881/900
188/188 [==============================] - 2s 8ms/step - loss: 0.0028 - v

In [72]:
yhat = model.predict(val_x)
new_val_x = val_x.reshape((val_x.shape[0], val_x.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, new_val_x[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
val_y = val_y.reshape((len(val_y), 1))
inv_y = concatenate((val_y, new_val_x[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

r2 = r2_score(inv_y, inv_yhat)
print('R-squared score: %.3f' % r2)

R-squared score: 0.926


In [34]:
scaled.shape

(17832, 27)

In [ ]:
# model.save("model_5.h5")

In [ ]:
#Investigating seasonality
subset = single_sensor[['date_time', 'hourly_counts']].copy()
subset.date_time = pd.to_datetime(subset.date_time)
pyplot.figure(figsize = (15, 5))
pyplot.plot(subset.date_time, subset.hourly_counts)

subset = single_sensor[['date_time', 'hourly_counts']].copy()
sub = subset.loc[(subset.date_time > '2021-12-31T23:00:00.000')].copy()
sub.date_time = pd.to_datetime(sub.date_time)
pyplot.figure(figsize = (15, 5))
pyplot.plot(sub.date_time, sub.hourly_counts)